# 가장 중요한 핵심 3가지
1. [url -> html]: resp = requests.get(url)
2. [html -> lxml]: soup = Beautifulsoup(resp.content,'lxml')
3. [lxml query]: tag = soup.select('tag1.class tag2')   [ .text ]  [ .strip() ]
- 원하는 태그를 불러온 후, .text나 .strip()을 적용 가능함